Camera Calibration with OpenCV
===

### The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

### Compute the camera calibration matrix and distortion coefficients given a set of chessboard images

In [ ]:
# Calibrate a camera, using a number of sample chessboard images
# Save the resulting 'dist' and 'mtx' values in a pickle file named
# 'calibration.p'

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import sys
import pickle
%matplotlib inline

def calibrate_camera():
    
    nx = 9
    ny = 6

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(nx-1,ny-1,0)
    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob("./camera_cal/calibration*.jpg")

    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        # If found, add object points, image points
        if ret == True:
            sys.stdout.write('+')
            objpoints.append(objp)
            imgpoints.append(corners)
        else:
            sys.stdout.write('-')
        
    # Test undistortion on an image
    img = cv2.imread('./camera_cal/calibration3.jpg')
    img_size = (img.shape[1], img.shape[0])

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump( dist_pickle, open( "./calibration.p", "wb" ) )
    
    return mtx, dist, img_size

# Using chessboard patterns, calibrate camera
mtx, dist, img_size = calibrate_camera()


### Display a sample chessboard calibration patter, alongside its undistorted transform

Step through a list of checkerboard patterns and display the original alongside the undistorated transform.

In [ ]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import sys
import pickle
%matplotlib inline

output_images_dir = "./output_images/"

def test_undist_patterns(mtx, dist, firstOnly=False, outputFile=False):

    try:
        os.mkdir(output_images_dir)
    except Exception:
        pass
    
    # Step through the list and undistort each image
    for fname in sorted(glob.glob(os.path.join('./camera_cal/', 'calibration*.jpg'))):
        img = cv2.imread(fname)

        # Apply anti-distortion transforms to the image
        dst = cv2.undistort(img, mtx, dist, None, mtx)

        # Visualize undistortion
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax1.set_title('Original Image', fontsize=30)
        ax2.imshow(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))
        ax2.set_title('Undistorted Image', fontsize=30)
        
        if outputFile:
            # Write each undistorted image to the 'output_images' dir
            cv2.imwrite(output_images_dir + 'undist_' + fname[11:], dst)
        
        if firstOnly:
            break
            
# Display undistorated checkerboard patterns
test_undist_patterns(mtx, dist, firstOnly=True, outputFile=True)

### Apply a distortion correction to (raw) test images

In [ ]:
import pickle
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os.path
%matplotlib inline

undist_images_dir = "./undist_images/"

def undistort_test_images(mtx, dist):
    
    # Make a list of calibration images
    test_images_dir = "./test_images/"

    try:
        os.mkdir(undist_images_dir)
    except Exception:
        pass

    # Step through the list and undistort each image
    for fname in sorted(glob.glob(os.path.join(test_images_dir, '*.jpg'))):
        img = cv2.imread(fname)

        # Apply anti-distortion transforms to the image
        dst = cv2.undistort(img, mtx, dist, None, mtx)

        # Write each undistorted image to the 'output_images' dir
        cv2.imwrite(undist_images_dir + fname[14:], dst)

        # Visualize undistortion
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax1.set_title('Original Image', fontsize=30)
        ax2.imshow(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))
        ax2.set_title('Undistorted Image', fontsize=30)        
            
    return output_images_dir

### Show test output
Show sample transformations that demonstrate how we calibrated the camera to remove distorion.

In [ ]:
# Using test_images, derive M and Minv for perspective transforms
undistort_test_images(mtx, dist)

### Apply color transforms, gradients, etc., to create a thresholded binary image

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os.path
%matplotlib inline
    
# Filter out the areas of the image where we don't expect to find
# lane lines    
def apply_trapezoid_mask(image, ignore_mask_color=255):
    
    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(image)   

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    xinset = 70
    xclip = 50
    yclip = 50
    vertices = np.array([[(xinset,imshape[0]),(imshape[1]/2-xclip, imshape[0]/2+yclip), (imshape[1]/2+xclip, imshape[0]/2+yclip), (imshape[1]-xinset,imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    return cv2.bitwise_and(image, mask)


# Define a function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
def abs_sobel_thresh(gray, orient='x', sobel_kernel=3, thresh=(0, 255)):
    
    # Apply the following steps to img
    
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # 5) Create a mask of 1's where the scaled gradient magnitude 
    #    is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    # 6) Return this mask as your binary_output image
    return binary_output


# Define a function that applies Sobel x and y, 
# then computes the magnitude of the gradient
# and applies a threshold
def mag_thresh(gray, sobel_kernel=3, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 2) Take the gradient in x and y separately
    # sobelx = abs_sobel_thresh(img, 'x', mag_thresh[0], mag_thresh[1])
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    # sobely = abs_sobel_thresh(img, 'y', mag_thresh[0], mag_thresh[1])
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    abs_sobelxy = np.sqrt(np.add(np.square(sobelx), np.square(sobely)))
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    # 5) Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output


# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # # Apply the following steps to img
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    graddir = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(graddir)
    binary_output[(graddir >= thresh[0]) & (graddir <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output


# Define a function that thresholds the S-channel of HLS
# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_select_s(img, thresh=(0, 255)):
    
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    # 2) Apply a threshold to the S channel
    S = hls[:,:,2]
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output


# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements
    

# Encapsulate a subset of the above transforms, after experimenting with various 
# combinations to discover what works best
def apply_threshold_transforms(img, showImages=False, imgName=None):
    
    # Convert color image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]

    # Apply each of the thresholding functions to gray
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(100, 255))
    dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0, np.pi/2))
        
#     sxbinary = np.zeros_like(gray)
#     sxbinary[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    sxbinary = gradx
        
    # Apply a threshold to the S channel
    s_binary = np.zeros_like(S)
    s_binary[(S > 100) & (S <= 120)] = 1
#     s_binary[(L > 100) & (L <= 255)] &= 1
    s_binary[(H > 80) & (H <= 255)] &= 1
        
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
#     stacked_binary = np.dstack(( np.zeros_like(gray), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    binary_image = np.zeros_like(sxbinary)
    binary_image[(s_binary == 1) | (sxbinary == 1)] = 1
    
    # Filter out the areas of the image where we don't expect to find
    # lane lines, leaving only a trapezoid around the receding lane lines
    binary_image = apply_trapezoid_mask(binary_image, 1)
    
    if imgName is not None:
        # Draw the trapezoid on the visualization image
        f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
        ax1.imshow(binary_image*255)
        ax1.set_title('Threshold Transform', fontsize=30)
        
        # Write each undistorted image to the 'output_images' dir
        cv2.imwrite(output_images_dir + 'threshold_' + imgName[16:], binary_image*255)
        
    
    return binary_image

def test_threshold():
    
    # Make a list of calibration images
    undist_images_dir = "./undist_images/"

    # Step through the list and undistort each image
    for fname in sorted(glob.glob(os.path.join(undist_images_dir, '*.jpg'))):
        dst = cv2.imread(fname)

        apply_threshold_transforms(dst, showImages=False, imgName=fname)
        
        break
        
test_threshold()

### Calculate the perspective transform

Once we have a thresholded binary image that isolates the lane line information, I warp it into a "birds-eye" view for further processing.

The values for providing the perspective transform were arrived at partially through a process of trial and error.

In [ ]:
import cv2

# Get M, the perspective transform for our camera.
def calc_perspective_transforms(img=None, imgName=None):
    
    src_pts = np.float32([[570,468],  [717,468], [1106,720], [207,720]])
    top_left = [320, 200]
    top_right = [920, 200]
    bottom_right = [920, 720]
    bottom_left = [320,720] 
    dst_pts = np.float32([top_left,top_right,bottom_right, bottom_left])

    if img is not None:
        # Draw the trapezoid on the visualization image
        pts = np.array([[570,468],  [717,468], [1106,720], [207,720]])
        pts = pts.reshape((-1,1,2))
        cv2.fillPoly(img,[pts],(0,255,255))
        f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
        ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax1.set_title('Perspective Transform Trapezoid', fontsize=30)
        
        # Write each undistorted image to the 'output_images' dir
        cv2.imwrite(output_images_dir + 'warp_pts_' + imgName[16:], img)

    
    # Calculate M and Minv
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    Minv = cv2.getPerspectiveTransform(dst_pts, src_pts)
    
    return M, Minv

M, Minv = calc_perspective_transforms()

def test_warp_pts():
    
    # Make a list of calibration images
    undist_images_dir = "./undist_images/"

    # Step through the list and undistort each image
    for fname in sorted(glob.glob(os.path.join(undist_images_dir, '*.jpg'))):
        dst = cv2.imread(fname)

        calc_perspective_transforms(dst, fname)
        
        break
        
test_warp_pts()

### Calculate the curvature
Derive the curvature, in meters
Estimate the number of meters per pixel for both the x and y axes of the 

In [ ]:
def calc_lane_curvature(leftx, lefty, rightx, righty):
    
    # Calculate curvature
    y_eval = 719  # 720p video/image, so last (lowest on screen) y index is 719

    # Define conversions in x and y from pixel space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in meters
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature, for both the Left and Right lane lines, in meters
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

#Alternative, for use during the fast-fit
def calc_lane_curvature2(left_lane_inds, right_lane_inds, nonzerox, nonzeroy):
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return calc_lane_curvature(leftx, lefty, rightx, righty)

### Calculate vehicle offset
Calculate whether the vehicle is to the Left or Right of the lane center,
and by how much, in meters

In [ ]:
def calc_vehicle_offset(undist, xm_per_pix, left_fit, right_fit):
    
    # Calculate vehicle center offset in pixels
    bottom_y = undist.shape[0] - 1
    bottom_x_left = left_fit[0]*(bottom_y**2) + left_fit[1]*bottom_y + left_fit[2]
    bottom_x_right = right_fit[0]*(bottom_y**2) + right_fit[1]*bottom_y + right_fit[2]
    vehicle_offset_m = undist.shape[1]/2 - (bottom_x_left + bottom_x_right)/2

    # Convert pixel offset to meters
    vehicle_offset_m *= xm_per_pix

    return vehicle_offset_m

### Apply a perspective transform to rectify binary image ("birds-eye view")

In [ ]:
# Return L and R Line objects
def apply_perspective_transform(binary_warped, M, showImages=False, imgName=None):
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 40
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If we found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    if showImages:
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        
            
    if imgName is not None:
        # Write each undistorted image to the 'output_images' dir
        cv2.imwrite(output_images_dir + 'perspective_' + imgName[16:], out_img)

            
    return left_fit, right_fit, leftx, lefty, rightx, righty

### Detect lane pixels and fit to find the lane boundary

In [ ]:
def get_warped_image(dst, M, showImages=False):
        
    warped_dst = cv2.warpPerspective(dst, M, img_size, flags=cv2.INTER_LINEAR)
    warped_orig = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    # Write the resulting image to the 'output_images' dir
#     gray_image = cv2.cvtColor(warped_dst*255, cv2.COLOR_GRAY2BGR)
#     cv2.imwrite(warped_images_dir + fname[16:], warped_dst)
    
    if showImages:
        # Plot the result
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        f.tight_layout()
        ax1.imshow(cv2.cvtColor(warped_orig, cv2.COLOR_BGR2RGB))
        ax1.set_title('Warped Image', fontsize=30)
        ax2.imshow(warped_dst, cmap='gray')
        ax2.set_title('Threshold Gradient', fontsize=30)
        
    return warped_dst

import copy

# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements

first = True

# Step through the list and undistort each image
for fname in sorted(glob.glob(os.path.join(undist_images_dir, '*.jpg'))):
    img = cv2.imread(fname)

    # Apply threshold transforms (color, sobel)
    binary_image = apply_threshold_transforms(img)

    # Get the warped image
    warped_dst = get_warped_image(binary_image, M, showImages=True)
    
    # Apply the perspective transform and generate a coded image showing
    # threshold detection, lane lines, and the boxes we used to detect 
    # key points in the lane lines
    if first:
        imgName = fname
        first = False
    else:
        imgName = None
    apply_perspective_transform(warped_dst, M, showImages=True, imgName=imgName)


In [ ]:
def anno_vid_image(undist, left_fit, right_fit, Minv, left_curve, right_curve, vehicle_offset):

    # Generate x and y values for plotting
    ploty = np.linspace(0, undist.shape[0]-1, undist.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw the lines on
    color_warp = np.zeros((img_size[1], img_size[0], 3), dtype='uint8')

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0]))
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    # Annotate lane curvature values and vehicle offset from center
    avg_curve = (left_curve + right_curve)/2
    label_str = 'Radius of curvature: %.1f m' % avg_curve
    result = cv2.putText(result, label_str, (50, 50), 0, 1, (0,0,0), 2, cv2.LINE_AA)

    if vehicle_offset < 0:
        label_str = 'Vehicle is %.1f m left of center' % -vehicle_offset
    else:
        label_str = 'Vehicle is %.1f m right of center' % vehicle_offset

    result = cv2.putText(result, label_str, (50, 80), 0, 1, (0,0,0), 2, cv2.LINE_AA)

    return result

### The Line class
Define a class to capture the characteristics of each line detection

In [ ]:
class Line():
    def __init__(self, n):
        
        self.n = n

        # Polynomial coefficients: x = A*y^2 + B*y + C
        # Each of A, B, C is a "list-queue" with max length n
        self.A = []
        self.B = []
        self.C = []
        # Average of above
        self.A_avg = 0.
        self.B_avg = 0.
        self.C_avg = 0.

    def get_fit(self):
        
        return (self.A_avg, self.B_avg, self.C_avg)

    def add_fit(self, fit_coeffs):

        # Coefficient queue full?
        q_full = len(self.A) >= self.n

        # Append line fit coefficients
        self.A.append(fit_coeffs[0])
        self.B.append(fit_coeffs[1])
        self.C.append(fit_coeffs[2])

        # Pop from index 0 if full
        if q_full:
            _ = self.A.pop(0)
            _ = self.B.pop(0)
            _ = self.C.pop(0)

        # Simple average of line coefficients
        self.A_avg = np.mean(self.A)
        self.B_avg = np.mean(self.B)
        self.C_avg = np.mean(self.C)

        return (self.A_avg, self.B_avg, self.C_avg)

### Write lane lines to the project video
For each frame in the video, apply all transforms and lane line detection, and save the resulting annotated frame to the annotated video file.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
from moviepy.editor import VideoFileClip

# Global variables (for use by annotate_frame())
with open('calibration.p', 'rb') as f:
    save_dict = pickle.load(f)
mtx = save_dict['mtx']
dist = save_dict['dist']

xm_per_pix = 3.7/700 # meters per pixel in x dimension

window_size = 5  # how many frames for line smoothing
left_line = Line(n=window_size)
right_line = Line(n=window_size)
detected = False  # did the fast line fit detect the lines?
left_curverad, right_curverad = 0., 0.  # radius of curvature for left and right lanes
left_lane_inds, right_lane_inds = None, None  # for calculating curvature

# Given a previously fit line, quickly try to find the line based on previous lines
def fast_fit(binary_warped, left_fit, right_fit):

    # Assume you now have a new warped binary image
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # If we don't find enough relevant points, return all None (this means error)
    min_inds = 12
    if lefty.shape[0] < min_inds or righty.shape[0] < min_inds:
        return None

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Return a dict of relevant variables
    ret = {}
    ret['left_fit'] = left_fit
    ret['right_fit'] = right_fit
    ret['nonzerox'] = nonzerox
    ret['nonzeroy'] = nonzeroy
    ret['left_lane_inds'] = left_lane_inds
    ret['right_lane_inds'] = right_lane_inds
    
    return ret

# MoviePy video annotation will call this function
def annotate_frame(img, showImages=False):
    
    global mtx, dist
    global left_line, right_line, detected
    global left_curverad, right_curverad, left_lane_inds, right_lane_inds

    # Apply anti-distortion transform to the image
    undist = cv2.undistort(img, mtx, dist, None, mtx)

    # Apply threshold transforms (color, sobel)
    binary_image = apply_threshold_transforms(undist)

    # Get the warped image
    warped_dst = get_warped_image(binary_image, M, showImages)
    
    # Perform polynomial fit
    if not detected:
        # Slow line fit
        print('Frame was not detected')
        
        # Apply the perspective transform and generate a coded image showing
        # threshold detection, lane lines, and the boxes we used to detect 
        # key points in the lane lines
        left_fit, right_fit, leftx, lefty, rightx, righty = apply_perspective_transform(warped_dst, M, showImages)

        # Get moving average of line fit coefficients
        left_fit = left_line.add_fit(left_fit)
        right_fit = right_line.add_fit(right_fit)

        # Calculate the curvature for each line, as radii in meters
        left_curverad, right_curverad = calc_lane_curvature(leftx, lefty, rightx, righty)

        detected = True  # slow lne fit always detects the line

    else:  # implies detected == True
        # Fast line fit

        left_fit = left_line.get_fit()
        right_fit = right_line.get_fit()
        ret = fast_fit(warped_dst, left_fit, right_fit)

        # Only make updates if we detected lines in current frame
        if ret is not None:
            left_fit = ret['left_fit']
            right_fit = ret['right_fit']
            nonzerox = ret['nonzerox']
            nonzeroy = ret['nonzeroy']
            left_lane_inds = ret['left_lane_inds']
            right_lane_inds = ret['right_lane_inds']

            left_fit = left_line.add_fit(left_fit)
            right_fit = right_line.add_fit(right_fit)
            left_curverad, right_curverad = calc_lane_curvature2(left_lane_inds, right_lane_inds, nonzerox, nonzeroy)
        else:
            print('Fit was not detected')
            detected = False

            
    vehicle_offset = calc_vehicle_offset(undist, xm_per_pix, left_fit, right_fit)

    # Perform final visualization on top of original undistorted image
    result = anno_vid_image(undist, left_fit, right_fit, Minv, left_curverad, right_curverad, vehicle_offset)

    return result

def annotate_test_frames(firstOnly=False, showImages=True):

    # Make a list of calibration images
    test_images_dir = "./test_images/"
    images = glob.glob(test_images_dir + "*.jpg")

    # Step through the list and undistort each image
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        
        # Apply anti-distortion transform to the image
        undist = cv2.undistort(img, mtx, dist, None, mtx)

        # Apply threshold transforms (color, sobel)
        binary_image = apply_threshold_transforms(undist)
        
        # Get the warped image
        warped_dst = get_warped_image(binary_image, M)

        # Apply the perspective transform and generate a coded image showing
        # threshold detection, lane lines, and the boxes we used to detect 
        # key points in the lane lines
        left_fit, right_fit, leftx, lefty, rightx, righty = apply_perspective_transform(warped_dst, M, showImages=False)

        # Calculate the curvature for each line, as radii in meters
        left_curverad, right_curverad = calc_lane_curvature(leftx, lefty, rightx, righty)
        vehicle_offset = calc_vehicle_offset(undist, xm_per_pix, left_fit, right_fit)

        # Perform final visualization on top of original undistorted image
        anno_img = anno_vid_image(undist, left_fit, right_fit, Minv, left_curverad, right_curverad, vehicle_offset)

        if showImages:
            # Visualize undistortion
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
            ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            ax1.set_title('Original Image', fontsize=30)
            ax2.imshow(cv2.cvtColor(anno_img, cv2.COLOR_BGR2RGB))
            ax2.set_title('Annotated Undistorted Image', fontsize=30)

        if firstOnly:
            break

def annotate_video(input_file, output_file):
    
    left_line = Line(n=window_size)
    right_line = Line(n=window_size)
    detected = False  # did the fast line fit detect the lines?
    left_curverad, right_curverad = 0., 0.  # radius of curvature for left and right lanes
    left_lane_inds, right_lane_inds = None, None  # for calculating curvature

    video = VideoFileClip(input_file)
    annotated_video = video.fl_image(annotate_frame)
    annotated_video.write_videofile(output_file, audio=False)
    
# Annotate the test frames first
# annotate_test_frames()

# Annotate the video
# annotate_video('project_video.mp4', 'project_out.mp4')

# Annotate the video
# annotate_video('challenge_video.mp4', 'challenge_out.mp4')

# Annotate the video
# annotate_video('harder_challenge_video.mp4', 'harder_challenge_out.mp4')

## 